In [112]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import joblib
from functools import reduce
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
#from skmultilearn.model_selection import MultiLabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

In [113]:
# breed
feat_breed = pd.read_pickle('../features/breed.pkl')
feat_breed_v2 = pd.read_pickle('../features/breed_v2.pkl')

feat_breed_top50 = pd.read_pickle('../features/breed_top50.pkl')
feat_breed_top50_v2 = pd.read_pickle('../features/breed_top50_v2.pkl')

feat_breed_group = pd.read_pickle('../features/breed_group.pkl')
feat_sub_breed = pd.read_pickle('../features/sub_breed.pkl')
feat_breed_type = pd.read_pickle('../features/breed_type.pkl')

feat_breed_pure_or_mix = pd.read_pickle('../features/breed_pure_or_mix.pkl')

# age
feat_age = pd.read_csv('../features/age_with_id.csv')

# sex
feat_sex = pd.read_csv('../features/one_hot_encoded_sex_with_id.csv')

# weight
feat_weight = pd.read_pickle('../features/weight.pkl')

# Climate
# feat_HotWheater = pd.read_csv('../features/one_hot_encoded_HotWheater_with_id.csv')
# feat_ModerateWheather = pd.read_csv('../features/one_hot_encoded_ModerateWheather_with_id.csv')
# feat_ColdWheater_with_id = pd.read_csv('../features/one_hot_encoded_ColdWheater_with_id.csv')
feat_cold_month = pd.read_pickle('../features/ColdMonths.pkl')
feat_hot_month = pd.read_pickle('../features/HotMonths.pkl')

# Diet
feat_diet = pd.read_pickle('../features/diet.pkl') # 5 columns

# Physical Activity
feat_pa_total_hours = pd.read_csv('../features/PhysicalActivity_total_hours.csv')
feat_pa_surface = pd.read_csv('../features/PhysicalActivity_surface.csv')
feat_pa_wheather = pd.read_csv('../features/PhysicalActivity_wheather.csv')

# Owner Demographics
feat_od_income = pd.read_pickle('../features/od_income.pkl')
feat_od_education = pd.read_pickle('../features/od_education.pkl')

# Residentual
feat_prim_census_division = pd.read_pickle('../features/primary_residence_census_division.pkl')

# disease
feat_disease_input = pd.read_csv('../features/one_hot_encoded_disease_input.csv')
feat_disease_output_binary = pd.read_csv('../features/disease_output_binary.csv')
feat_disease_output = pd.read_csv('../features/disease_output.csv')

# age_condition
feat_age_condition = pd.read_pickle('../features/age_condition.pkl')
feat_age_condition_type = pd.read_pickle('../features/age_condition_type.pkl')


features_list = [

    # breed
    #feat_breed,
    #feat_breed_v2,
    #feat_breed_top50,
    #feat_breed_top50_v2,
    #feat_breed_group,
    #feat_sub_breed,
    #feat_breed_type,
    #feat_breed_pure_or_mix,

    # age
    #feat_age,   # 24881 dog_ids

    # sex
    #feat_sex,

    # weight
    #feat_weight

    # diet
    #feat_diet,  # 33141 dog_ids for df_diet_consistency

    #feat_HotWheater,
    #feat_ModerateWheather,
    #feat_ColdWheater_with_id,

    # Climate
    #feat_pa_surface,
    #feat_pa_total_hours,
    #feat_pa_wheather,   # 26406 dog_ids
    #feat_hot_month,
    #feat_cold_month,

    # Physical Activity
    #feat_pa_total_hours,
    #feat_pa_surface,

    # Owner demographics
    #feat_od_income,    # 29096 dog_ids
    #feat_od_education

    # Residentual
    feat_prim_census_division,  # 33136 dog_ids

    # disease
    #feat_disease_input,
    #feat_disease_output_binary,
    #feat_disease_output
]

feat_age_condition_type

,dog_id,age_diagnosis_years,condition_type_Bone/Orthopedic,condition_type_Brain/Neurologic,condition_type_Cardiac,condition_type_Ear/Nose/Throat,condition_type_Endocrine,condition_type_Eye,condition_type_Gastrointestinal,condition_type_Hematopoietic,...,condition_type_Kidney/Urinary,condition_type_Liver/Pancreas,condition_type_Mouth/Dental/Oral,condition_type_Other Congenital Disorder,condition_type_Reproductive,condition_type_Respiratory,condition_type_Skin,condition_type_Toxin Consumption,condition_type_Trauma,condition_type_cancer
0,100001,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,100001,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,100001,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,100001,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,100001,4,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910915,3373,10,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910929,14856,10,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910936,58327,6,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
910945,3417,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [114]:
exp1 = [feat_breed_group]
exp2 = [feat_breed_top50_v2]
exp3 = [feat_age]
exp4 = [feat_sex,feat_breed_type]
exp5 = [feat_sub_breed]
exp6 = [feat_breed_pure_or_mix]
exp7 = [feat_breed_top50_v2,feat_breed_group,feat_weight]
exp8 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex]
exp9 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather]
exp10 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division]
exp11 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_od_education]
exp12 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather]
exp13 = [feat_breed_top50_v2,feat_breed_group,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather]
exp14 = [feat_breed_top50_v2,feat_breed_group,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_diet]
exp15 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_diet]
exp16 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_cold_month,feat_hot_month,feat_prim_census_division,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet]
exp17 = [feat_breed_top50_v2,feat_breed_group,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_age,feat_sex,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet,feat_weight]
exp18 = [feat_breed_top50_v2,feat_breed_group,feat_od_income,feat_od_education,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet]
exp19 = [feat_breed_top50_v2,feat_breed_group,feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,feat_sub_breed,feat_diet,feat_weight]
exp20 = [feat_breed_top50_v2,feat_breed_group,feat_sub_breed,
         feat_od_education, feat_od_income,
         feat_cold_month,feat_hot_month,
         feat_prim_census_division,
         feat_pa_surface,feat_pa_total_hours,feat_pa_wheather,
         feat_breed_pure_or_mix,feat_breed_type,feat_weight,feat_sex,feat_age
         ]

In [115]:
experiments = {
    'exp1': exp1,
    'exp2': exp2,
    'exp3': exp3,
    'exp4': exp4,
    'exp5': exp5,
    'exp6': exp6,
    'exp7': exp7,
    'exp8': exp8,
    'exp9': exp9,
    'exp10': exp10,
    'exp11': exp11,
    'exp12': exp12,
    'exp13': exp13,
    'exp14': exp14,
    'exp15': exp15,
    'exp16': exp16,
    'exp17': exp17,
    'exp18': exp18,
    'exp19': exp19,
    'exp20': exp20,
}


In [116]:
feat_prim_census_division

,dog_id,oc_primary_census_division__1.0,oc_primary_census_division__2.0,oc_primary_census_division__3.0,oc_primary_census_division__4.0,oc_primary_census_division__5.0,oc_primary_census_division__6.0,oc_primary_census_division__7.0,oc_primary_census_division__8.0,oc_primary_census_division__9.0
0,10509,False,False,False,True,False,False,False,False,False
1,74227,False,False,False,False,True,False,False,False,False
2,32487,False,False,False,False,True,False,False,False,False
3,63150,False,False,False,False,False,False,False,False,True
4,33369,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
33167,91849,False,False,False,False,False,False,False,False,True
33168,33783,False,False,False,False,False,False,False,False,True
33169,98862,False,False,False,False,False,False,True,False,False
33170,99462,False,False,False,False,False,False,False,False,True


In [117]:
# List of DataFrames to be merged
list_input_features = features_list  # Add more DataFrames as needed

# Merge DataFrames iteratively using reduce
input_features = reduce(lambda left, right: pd.merge(left, right, on='dog_id'), exp20)

# Merge with disease output feature
# data = pd.merge(input_features, feat_disease_output_binary, on='dog_id')
data = pd.merge(feat_disease_output_binary, input_features, on='dog_id')
data

,dog_id,hs_health_conditions_eye,hs_health_conditions_ear,hs_health_conditions_oral,hs_health_conditions_skin,hs_health_conditions_cardiac,hs_health_conditions_respiratory,hs_health_conditions_gastrointestinal,hs_health_conditions_liver,hs_health_conditions_kidney,...,breed_type_Toy dogs,breed_type_Working dogs,dd_weight_lbs,sex_1,sex_2,rounded_age_years,od_annual_income_range_usd,pa_hot_weather_daily_hours_outside,pa_cold_weather_daily_hours_outside,pa_moderate_weather_daily_hours_outside
0,10509,0,0,0,1,0,0,0,0,0,...,False,False,33.0,False,True,16.0,4.0,1.0,1.0,1.0
1,74227,0,0,0,1,0,0,0,0,0,...,False,False,44.0,False,True,2.0,10.0,1.0,1.0,2.0
2,33369,0,0,0,0,0,0,0,0,0,...,False,False,34.8,True,False,2.0,7.0,1.0,2.0,2.0
3,72814,0,0,1,0,0,0,0,0,1,...,False,False,91.0,False,True,5.0,4.0,1.0,1.0,1.0
4,72448,0,0,1,0,1,0,0,0,0,...,False,False,15.0,False,True,11.0,3.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19164,46957,0,0,0,0,0,0,0,0,0,...,False,False,31.0,False,True,8.0,4.0,1.0,2.0,2.0
19165,89162,1,0,0,0,0,0,0,0,0,...,False,False,62.0,True,False,11.0,6.0,1.0,1.0,2.0
19166,89372,1,0,1,1,0,0,0,0,0,...,False,False,11.7,True,False,4.0,10.0,1.0,1.0,1.0
19167,12758,0,0,1,0,0,0,0,1,0,...,False,False,8.0,False,True,10.0,8.0,1.0,1.0,1.0


In [118]:
def calculate_average_auc_scores(auc_scores_per_condition, sample_counts_per_condition):
    num_folds = len(list(auc_scores_per_condition.values())[0])
    average_auc_scores_per_fold = []

    for fold in range(num_folds):
        weighted_aucs = []

        for condition_type in auc_scores_per_condition.keys():
            auc_scores_fold = auc_scores_per_condition[condition_type][fold]
            sample_counts_fold = sample_counts_per_condition[condition_type][fold]

            weighted_auc = auc_scores_fold * sample_counts_fold
            #print(auc_scores_fold, f"sample_counts[{condition_type}][{fold}]", sample_counts_fold)
            weighted_aucs.append(weighted_auc)

        total_weighted_auc = np.sum(weighted_aucs)
        total_samples = np.sum([sample_counts_per_condition[condition_type][fold] for condition_type in auc_scores_per_condition.keys()])
        average_auc_score_fold = total_weighted_auc / total_samples
        average_auc_scores_per_fold.append(average_auc_score_fold)

    return average_auc_scores_per_fold

In [119]:
results_auc_train = {}
results_auc_val = {}
results_accuracy = {}

for exp_name, exp_features in experiments.items():

    print("======================================================================================")
    print("======================================================================================")
    print(f"{exp_name}================================================================================")
    print("======================================================================================")
    print("======================================================================================")

    # List of DataFrames to be merged
    list_input_features = features_list  # Add more DataFrames as needed

    # Merge DataFrames iteratively using reduce
    input_features = reduce(lambda left, right: pd.merge(left, right, on='dog_id'), exp_features)

    # Merge with disease output feature
    # data = pd.merge(input_features, feat_disease_output_binary, on='dog_id')
    data = pd.merge(feat_disease_output_binary, input_features, on='dog_id')

    # Normalize the specified features in one line and save in the same columns
    features_to_normalize = ['dd_weight_lbs', 'age_diagnosis_years', 'rounded_age_years', 'df_diet_consistency', 'df_feedings_per_day', 'total_active_hours_y', 'pa_hot_weather_months_per_year', 'pa_cold_weather_months_per_year', 'od_max_education', 'od_annual_income_range_usd', 'pa_moderate_weather_daily_hours_outside', 'pa_cold_weather_daily_hours_outside', 'pa_cold_weather_daily_hours_outside']

    # select features that are in data
    features_to_normalize_in_data = [feature for feature in features_to_normalize if feature in data.columns]

    # Initialize MinMaxScaler
    scaler = MinMaxScaler()

    # Normalize
    if features_to_normalize_in_data:
        data[features_to_normalize_in_data] = scaler.fit_transform(data[features_to_normalize_in_data])
        # print(data[features_to_normalize_in_data])

    X = data.drop(['dog_id'] + ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]], axis=1)

    y_columns = ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]]

    y = data[y_columns]

    # Convert y to a binary format
    y_binary = (y == 1)

    ############################################################################################################################
    ######## SELECT MODEL ######################################################################################################
    ############################################################################################################################

    #select_model = "Naive Bayes"
    #select_model = "Logistic Regression"
    #select_model = "MLP"
    select_model = "Extreme Gradien Boosting"


    if select_model=="Naive Bayes":
    # ========== Naive Bayes ========
        # Initialize the Naive Bayes model
        nb_model = MultinomialNB()

        # Wrap the model with OneVsRestClassifier
        ovr_classifier = OneVsRestClassifier(nb_model)

    elif select_model=="Logistic Regression":
        # Initialize the Logistic Regression model
        lr_model = LogisticRegression()

        # Wrap the model with OneVsRestClassifier
        ovr_classifier = OneVsRestClassifier(lr_model)

    elif select_model=="MLP":
    # ========= MLP ==========
        # normalize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        num_features = X.shape[1]
        if num_features <= 100:
                hidden_layer_sizes = (60,)
        elif num_features <= 200:
                hidden_layer_sizes = (60, 40)
        else:
            hidden_layer_sizes = (80, 60, 40)

        # Initialize the MLP model
        mlp_model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=1000, alpha=0.01, early_stopping=True, random_state=42)
        # Wrap the model with OneVsRestClassifier
        ovr_classifier = OneVsRestClassifier(mlp_model)

    elif select_model=="Extrem Gradient boosting":
        # Initialize the Extreme Gradient Boosting model
        model = xgb.XGBClassifier()

        # Wrap the model with OneVsRestClassifier
        ovr_classifier = OneVsRestClassifier(model)

    # Initialize MultilabelStratifiedKFold
    n_splits = 5  # You can adjust the number of splits as needed
    ml_stratified_kfold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Lists to store AUC scores and sample counts
    auc_scores_train_per_condition = {condition: [] for condition in y_columns}
    auc_scores_val_per_condition = {condition: [] for condition in y_columns}
    sample_counts_train_per_condition = {condition: [] for condition in y_columns}
    sample_counts_val_per_condition = {condition: [] for condition in y_columns}
    auc_scores_val_per_fold = []
    auc_scores_train_per_fold = []

    accuracy_scores_per_fold = []
    auc_scores_per_fold_train = []
    auc_scores_per_fold_val = []
    # Lists to store AUC scores and sample counts
    auc_scores_per_condition = {condition: [] for condition in y_columns}
    sample_counts_per_condition = {condition: [] for condition in y_columns}

    # Iterate through the splits
    for fold, (train_index, val_index) in enumerate(ml_stratified_kfold.split(X, y_binary)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[val_index]

        # Train the model
        ovr_classifier.fit(X_train, y_train)

        # Make predictions on the training set
        y_train_pred_proba = ovr_classifier.predict_proba(X_train)

        # Make predictions on the validation set
        y_val_pred_proba = ovr_classifier.predict_proba(X_val)


        # Calculate the AUC score for each disease
        auc_scores_train = [roc_auc_score(y_train[column], y_train_pred_proba[:, i]) for i, column in enumerate(y_train.columns)]
        auc_scores_val = [roc_auc_score(y_val[column], y_val_pred_proba[:, i]) for i, column in enumerate(y_val.columns)]

        # Calculate AUC score for each disease
        fold_auc_val = roc_auc_score(y_val.values.ravel(), y_val_pred_proba.ravel())
        auc_scores_val_per_fold.append(fold_auc_val)

        fold_auc_train = roc_auc_score(y_train.values.ravel(), y_train_pred_proba.ravel())
        auc_scores_train_per_fold.append(fold_auc_train)

        # get sample counts
        for i, condition in enumerate(y_columns):
            sample_count = y_train[condition].sum()  # Count of positive samples for the condition
            sample_counts_train_per_condition[condition].append(sample_count)

        for i, condition in enumerate(y_columns):
            sample_count = y_val[condition].sum()  # Count of positive samples for the condition
            sample_counts_val_per_condition[condition].append(sample_count)

        print("EXPERIMENT: ", exp_name)
        print(f"Fold {fold+1} ========================================")
        #print(f"\nFold {fold+1} AUC Scores for Diseases TRAINING-set:")
        for i, auc_score in enumerate(auc_scores_train, start=1):
            #print(f"{y_train.columns[i-1]}: {auc_score}")
            auc_scores_train_per_condition[y_train.columns[i-1]].append(auc_score)

        #print(f"\nFold {fold+1} AUC Scores for Diseases VALIDATION-set:")
        for i, auc_score in enumerate(auc_scores_val, start=1):
            #print(f"{y_val.columns[i-1]}: {auc_score}")
            auc_scores_val_per_condition[y_val.columns[i-1]].append(auc_score)

        if exp_name == 'exp20':
            exp20_auc_scores_per_condition = auc_scores_per_condition.copy()


    # Calculate average AUC scores for training and validation sets
    average_auc_scores_per_fold_train = calculate_average_auc_scores(auc_scores_train_per_condition, sample_counts_train_per_condition)
    #print("==========")
    average_auc_scores_per_fold_val = calculate_average_auc_scores(auc_scores_val_per_condition, sample_counts_val_per_condition)

    # Print the results
    #for i, average_auc in enumerate(average_auc_scores_per_fold_train, start=1):
        #print(f'AUC-Score Train Fold-{i}: {average_auc}')

    #for i, average_auc in enumerate(average_auc_scores_per_fold_val, start=1):
        #print(f'AUC-Score Val Fold-{i}: {average_auc}')

    # Calculate and print the total average AUC score and the maximum difference for training set
    total_average_auc_train = np.mean(average_auc_scores_per_fold_train)
    max_difference_train = np.max(np.abs(average_auc_scores_per_fold_train - total_average_auc_train))
    #print(f'Total Average AUC Score train: {total_average_auc_train * 100:.2f} ± {max_difference_train * 100:.2f}%')

    # Calculate and print the total average AUC score and the maximum difference for validation set
    total_average_auc_val = np.mean(average_auc_scores_per_fold_val)
    max_difference_val = np.max(np.abs(average_auc_scores_per_fold_val - total_average_auc_val))
    #print(f'Total Average AUC Score val: {total_average_auc_val * 100:.2f} ± {max_difference_val * 100:.2f}%')


    # Calculate average AUC per condition for VAL
    average_auc_val_per_condition = {
        condition: sum(auc_scores) / len(auc_scores) for condition, auc_scores in auc_scores_val_per_condition.items()
    }
    # Calculate overall average AUC weighted by the number of samples
    overall_average_auc_val_sum = 0
    for auc_scores, sample_counts in zip(auc_scores_val_per_condition.values(), sample_counts_val_per_condition.values()):
        weighted_sum = sum(auc * sample_count for auc, sample_count in zip(auc_scores, sample_counts))
        overall_average_auc_val_sum += weighted_sum / sum(sample_counts)

    overall_average_auc_val = overall_average_auc_val_sum / len(auc_scores_val_per_condition)  # divide by the number of conditions

    # Calculate average AUC per condition for TRAIN
    average_auc_train_per_condition = {
        condition: sum(auc_scores) / len(auc_scores) for condition, auc_scores in auc_scores_train_per_condition.items()
    }
    # Calculate overall average AUC weighted by the number of samples
    overall_average_auc_train_sum = 0
    for auc_scores, sample_counts in zip(auc_scores_train_per_condition.values(), sample_counts_train_per_condition.values()):
        weighted_sum = sum(auc * sample_count for auc, sample_count in zip(auc_scores, sample_counts))
        overall_average_auc_train_sum += weighted_sum / sum(sample_counts)

    overall_average_auc_train = overall_average_auc_train_sum / len(auc_scores_train_per_condition)  # divide by the number of conditions


    # Average AUC-score per condition
    #print("\n Average AUC train-score per condition:")
    #for condition, avg_auc in average_auc_train_per_condition.items():
    #    print(f"{condition}: {avg_auc}")

    print("\n Average AUC val-score per condition:")
    for condition, avg_auc in average_auc_val_per_condition.items():
        print(f"{condition}: {avg_auc}")

    # Get average AUC values as a list
    average_auc_val_list = list(average_auc_val_per_condition.values())
    average_auc_train_list = list(average_auc_train_per_condition.values())

    # Print average AUC scores
    print("\nWeighted Average AUC Score:")
    #print(f"Overall: {overall_average_auc}")
    print(f"Train: {overall_average_auc_train * 100:.2f} ± {max_difference_train * 100:.2f}%")
    print(f"Val: {overall_average_auc_val * 100:.2f} ± {max_difference_val * 100:.2f}%")

    results_auc_train[exp_name] = f"Train: {overall_average_auc_train * 100:.2f} ± {max_difference_train * 100:.2f}%"
    results_auc_val[exp_name] = f"Val: {overall_average_auc_val * 100:.2f} ± {max_difference_val * 100:.2f}%"

exp1================================================================================
EXPERIMENT:  exp1
Fold 1 ========================================
EXPERIMENT:  exp1
Fold 2 ========================================
EXPERIMENT:  exp1
Fold 3 ========================================
EXPERIMENT:  exp1
Fold 4 ========================================
EXPERIMENT:  exp1
Fold 5 ========================================

 Average AUC val-score per condition:
hs_health_conditions_eye: 0.5257769466362794
hs_health_conditions_ear: 0.5416119288049387
hs_health_conditions_oral: 0.6311794207245731
hs_health_conditions_skin: 0.5335511743988868
hs_health_conditions_cardiac: 0.5300022723603706
hs_health_conditions_respiratory: 0.4643280536469662
hs_health_conditions_gastrointestinal: 0.4903649965218195
hs_health_conditions_liver: 0.46674954467705926
hs_health_conditions_kidney: 0.49797034969419957
hs_health_conditions_reproductive: 0.5036593486675428
hs_health_conditions_orthopedic: 0.5133166063750652
h

In [120]:
# Print the results
print("Training Set AUC Scores:")
for exp_name, auc_score in results_auc_train.items():
    print(f"{exp_name}: Average AUC = {auc_score}")

print("\nValidation Set AUC Scores:")
for exp_name, auc_score in results_auc_val.items():
    print(f"{exp_name}: Average AUC = {auc_score}")

# for exp_name, accuracy_score in results_accuracy.items():
#     print(f"{exp_name}: Average Accuracy = {accuracy_score:.4f}")


Training Set AUC Scores:
exp1: Average AUC = Train: 51.98 ± 0.14%
exp2: Average AUC = Train: 57.26 ± 0.52%
exp3: Average AUC = Train: 42.93 ± 1.69%
exp4: Average AUC = Train: 55.81 ± 0.16%
exp5: Average AUC = Train: 55.39 ± 0.05%
exp6: Average AUC = Train: 52.15 ± 0.13%
exp7: Average AUC = Train: 58.37 ± 0.21%
exp8: Average AUC = Train: 61.28 ± 1.82%
exp9: Average AUC = Train: 62.55 ± 1.80%
exp10: Average AUC = Train: 58.45 ± 1.71%
exp11: Average AUC = Train: 60.34 ± 1.95%
exp12: Average AUC = Train: 62.05 ± 1.18%
exp13: Average AUC = Train: 58.67 ± 1.46%
exp14: Average AUC = Train: 64.21 ± 2.58%
exp15: Average AUC = Train: 64.33 ± 2.96%
exp16: Average AUC = Train: 61.74 ± 1.55%
exp17: Average AUC = Train: 64.85 ± 0.68%
exp18: Average AUC = Train: 61.17 ± 0.99%
exp19: Average AUC = Train: 61.66 ± 0.56%
exp20: Average AUC = Train: 64.31 ± 0.96%

Validation Set AUC Scores:
exp1: Average AUC = Val: 51.66 ± 0.30%
exp2: Average AUC = Val: 56.10 ± 0.41%
exp3: Average AUC = Val: 42.92 ± 1.47%

In [108]:
##  MLP
#Training Set AUC Scores:
#exp1: Average AUC = Train: 52.40 ± 0.21%
#exp2: Average AUC = Train: 54.97 ± 0.75%
#exp3: Average AUC = Train: 38.60 ± 2.04%
#exp4: Average AUC = Train: 53.19 ± 0.08%
#exp5: Average AUC = Train: 52.57 ± 0.18%
#exp6: Average AUC = Train: 51.60 ± 0.36%
#exp7: Average AUC = Train: 56.49 ± 0.80%
#exp8: Average AUC = Train: 56.15 ± 1.72%
#exp9: Average AUC = Train: 58.25 ± 1.83%
#exp10: Average AUC = Train: 55.43 ± 2.59%
#exp12: Average AUC = Train: 58.71 ± 1.97%
#exp13: Average AUC = Train: 58.68 ± 0.69%
#exp14: Average AUC = Train: 62.77 ± 0.75%
#exp15: Average AUC = Train: 64.33 ± 2.96%
#exp16: Average AUC = Train: 61.74 ± 1.55%
#exp11: Average AUC = Train: 56.13 ± 1.46%
#exp17: Average AUC = Train: 64.85 ± 0.68%
#exp18: Average AUC = Train: 61.17 ± 0.99%
#exp19: Average AUC = Train: 61.66 ± 0.56%
#exp20: Average AUC = Train: 64.31 ± 0.96%
#
#Validation Set AUC Scores:
#exp1: Average AUC = Val: 52.17 ± 0.43%
#exp2: Average AUC = Val: 54.47 ± 0.40%
#exp3: Average AUC = Val: 38.56 ± 1.73%
#exp4: Average AUC = Val: 53.05 ± 0.74%
#exp5: Average AUC = Val: 52.43 ± 0.90%
#exp6: Average AUC = Val: 51.62 ± 0.50%
#exp7: Average AUC = Val: 55.76 ± 0.46%
#exp8: Average AUC = Val: 55.20 ± 1.79%
#exp9: Average AUC = Val: 57.19 ± 1.68%
#exp10: Average AUC = Val: 54.39 ± 2.26%
#exp11: Average AUC = Val: 55.44 ± 2.42%
#exp12: Average AUC = Val: 57.38 ± 2.03%
#exp13: Average AUC = Val: 57.37 ± 0.93%
#exp14: Average AUC = Val: 60.85 ± 0.97%
#exp15: Average AUC = Val: 61.74 ± 1.52%
#exp16: Average AUC = Val: 59.58 ± 1.76%
#exp17: Average AUC = Val: 62.40 ± 0.32%
#exp18: Average AUC = Val: 58.77 ± 0.41%
#exp19: Average AUC = Val: 59.94 ± 0.68%
#exp20: Average AUC = Val: 61.74 ± 0.48%

SyntaxError: invalid character '±' (U+00B1) (1516980268.py, line 2)

In [108]:
## Extreme gradient boosting
#Training Set AUC Scores:
#exp1: Average AUC = Train: 51.98 ± 0.14%
#exp2: Average AUC = Train: 57.26 ± 0.52%
#exp3: Average AUC = Train: 42.93 ± 1.69%
#exp4: Average AUC = Train: 55.81 ± 0.16%
#exp5: Average AUC = Train: 55.39 ± 0.05%
#exp6: Average AUC = Train: 52.15 ± 0.13%
#exp7: Average AUC = Train: 58.37 ± 0.21%
#exp8: Average AUC = Train: 61.28 ± 1.82%
#exp9: Average AUC = Train: 62.55 ± 1.80%
#exp10: Average AUC = Train: 58.45 ± 1.71%
#exp11: Average AUC = Train: 60.34 ± 1.95%
#exp12: Average AUC = Train: 62.05 ± 1.18%
#exp13: Average AUC = Train: 58.67 ± 1.46%
#exp14: Average AUC = Train: 64.21 ± 2.58%
#exp15: Average AUC = Train: 64.33 ± 2.96%
#exp16: Average AUC = Train: 61.74 ± 1.55%
#exp17: Average AUC = Train: 64.85 ± 0.68%
#exp18: Average AUC = Train: 61.17 ± 0.99%
#exp19: Average AUC = Train: 61.66 ± 0.56%
#exp20: Average AUC = Train: 64.31 ± 0.96%
#
#Validation Set AUC Scores:
#exp1: Average AUC = Val: 51.66 ± 0.30%
#exp2: Average AUC = Val: 56.10 ± 0.41%
#exp3: Average AUC = Val: 42.92 ± 1.47%
#exp4: Average AUC = Val: 55.04 ± 0.16%
#exp5: Average AUC = Val: 54.79 ± 0.33%
#exp6: Average AUC = Val: 52.15 ± 0.32%
#exp7: Average AUC = Val: 56.79 ± 0.33%
#exp8: Average AUC = Val: 59.40 ± 1.27%
#exp9: Average AUC = Val: 60.58 ± 0.84%
#exp10: Average AUC = Val: 56.64 ± 1.34%
#exp11: Average AUC = Val: 58.42 ± 0.93%
#exp12: Average AUC = Val: 60.02 ± 1.10%
#exp13: Average AUC = Val: 57.02 ± 0.66%
#exp14: Average AUC = Val: 61.66 ± 1.28%
#exp15: Average AUC = Val: 61.74 ± 1.52%
#exp16: Average AUC = Val: 59.58 ± 1.76%
#exp17: Average AUC = Val: 62.40 ± 0.32%
#exp18: Average AUC = Val: 58.77 ± 0.41%
#exp19: Average AUC = Val: 59.94 ± 0.68%
#exp20: Average AUC = Val: 61.74 ± 0.48%